In [ ]:
import matplotlib.pyplot as plt # For plotting
import numpy as np              # Linear algebra library
import pandas as pd

In [ ]:
! pwd
expr_df = pd.read_csv("../metadata/length_and_depth.csv")
expr_df = expr_df.drop("nvar", axis=1)
expr_df = expr_df.rename(columns={
    "length" : "expr_length",
    "depth" : "expr_depth"
})
expr_df

In [ ]:
complete_df = pd.read_csv("../results/complete_dataset_as_of_nov6.csv")
complete_df = complete_df[complete_df["is_init_run"] == 0]
complete_df = complete_df.rename(columns={"name" : "problem"})
df = complete_df.merge(
    expr_df,
    on = 'problem',
    how = "inner"
)
df

In [ ]:
problems = df["problem"].unique()
rng = np.random.default_rng(seed=42)
rng.shuffle(problems)

n = len(problems)
n_train = int(0.7 * n)
n_valid = int(0.15 * n)

train_problems = problems[:n_train]
valid_problems = problems[n_train:n_train + n_valid]
test_problems  = problems[n_train + n_valid:]

train_df = df[df["problem"].isin(train_problems)].reset_index(drop=True)
valid_df = df[df["problem"].isin(valid_problems)].reset_index(drop=True)
test_df  = df[df["problem"].isin(test_problems)].reset_index(drop=True)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
feature_cols = ["nvar", 
                "expr_length", 
                "expr_depth", 
                "mem",
                "init_eval_obj_time", 
                "init_eval_obj_mem",
                "init_eval_obj_alloc",
                "init_eval_grad_time",
                "init_eval_grad_mem",
                "init_eval_grad_alloc"]
target_col = "time"
X_train = train_df[feature_cols].to_numpy(dtype=float)
X_valid = valid_df[feature_cols].to_numpy(dtype=float)
X_test  = test_df[feature_cols].to_numpy(dtype=float)

t_train = np.log(train_df[target_col].to_numpy(dtype=float))
t_valid = np.log(valid_df[target_col].to_numpy(dtype=float))
t_test  = np.log(test_df[target_col].to_numpy(dtype=float))